In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os

# Purpose and scope
This notebook will tell you how to set up and use [matplotlib](https://matplotlib.org) together with [TikZ](https://en.wikipedia.org/wiki/PGF/TikZ) (requires LaTeX installation) to create publication ready figures.  
You will learn to:
1. define a matplotlib style sheet for ensuring that all figure panels follow the same style (font sizes, line widths, markers, etc.), 
1. create a python module with functions for creating matplotlib figures and axes of a specific size, 
1. use the module to create and store figure panels in (preferably) vectorized formats (e.g. pdf),
1. combine individual figure panels into a complete figure using TikZ.

### Creating a complete figure usually requires two softwares
A complete figure usually consist of multiple figure panels (the elements normally labeled a, b, c, ...) and possible additional graphical elements. The most common panel type in scientific articles and reports is the data panel (2D or 3D), that is, various types of graphs and charts, but illustrations are also common examples. 

Data panels are typically created using the same tools used for analyzing the data, for example, python or Matlab. However, these tools are not well suited for producing general graphical content, nor are they very good at combining multiple panels into a complete figure (unless it is a simple figure containing only data panels). You therefore need to master at least one additional software to produce more general graphical content. Free examples of such softwares include [Inkscape](https://inkscape.org) and [TikZ](https://en.wikipedia.org/wiki/PGF/TikZ), while commercial software examples are Photoshop and PainShop Pro. 

As outlined above, this guide will focus on python (using matplotlib) and TikZ. The reasons being that 1) these are free, 2) they are text (code) based and thus fully compatible with Git, and 3) they don't require a graphical user interface, meaning that complete figures can be created from scratch using terminal commands only. Taken together, these benefits let you keep everything needed for analyzing your data and for producing the figures in your publication in one git repository, and all commands needed for analyzing and generating your figures can be collected in one script. Thus, making it very easy to comply with open access policies as anybody can reproduce your publication figures once the repository is made public and assuming that the raw data can be shared through, for example, [Zenodo](https://zenodo.org).

### Before getting started
Before even trying to create a nice publication ready figure you need to think through what you want to plot and how. The two most important questions are:
1. The overall layout of your figure, that is, how many figure panels do you want to include and how should they roughly be organized?
1. How much space do you have at your disposal, or how large do you want the figure to be?

You should always create the figure to be of the correct size right away. If you start resizing it later, you are very likely to loose track of the final font sizes and line widths, with the end result being that your figures or figure panels don't look like they belong together. As a general starting point, you can assume that a figure intended to cover the full width of an A4 paper should be 16 to 18 cm wide, and roughly half that for a figure covering the full width of one column in a two-column format.

In this guide, we will aim to create a figure about interpolation techniques (linear vs. cubic). Imagine two smaller data panels for showing a linear and a cubic function, and one large panel for illustrating the difference between linear and cubic interpolation. The complete figure should match the text width in an A4 paper and we thus aim for a figure width of roughly 16 cm.

### Preparing directories for files to be created
This notebook will produce several files and we will therefore add a couple of directories to store things in (unless they already exist).

In [ ]:
figure_panel_dir = 'Figure panels'
if not os.path.isdir(figure_panel_dir):
    os.makedirs(figure_panel_dir)
if not os.path.isdir('Tikz'):
    os.makedirs('Tikz')

# Part 1, creating figure panels with matplotlib

### Defining the style sheet
The matplotlib style sheet defines default values for essentially all things that you can adjust when plotting. You should define your own style sheet where you set default values for elements that will occur in many places (e.g. font sizes, line widths, etc.). The cell below includes a template that will get saved once you run the cell (observe the IPython cell magic command, %%writefile one the first row).

In [ ]:
%%writefile default_style.mplstyle
# Style sheet for defining standard values to be used by matplotlib
#
# See:
# https://matplotlib.org/stable/tutorials/introductory/customizing.html
# for a full list of all things that can be set

# Font sizes
# A default font size of 8 to 10 pt is good for figures intended for articles.
# For figure that will be shown on presentations you should use at least 12 or 14 pt.
#
# Relative font sizes: 
# ('xx-small', 'x-small', 'small', 'medium', 'large', 'x-large', 'xx-large')
# 'xx-small' corressponds to 1.2**-3 * font.size
# 'medium' corressponds to 1.2**0 * font.size
# 'xx-large' corressponds to 1.2**3 * font.size
font.size:        10
axes.titlesize:   large
axes.labelsize:   medium
xtick.labelsize:  medium
ytick.labelsize:  medium
legend.fontsize:  medium

# Axes
# Spines (black lines around the axes)
axes.spines.left:   True
axes.spines.bottom: True
axes.spines.top:    False
axes.spines.right:  False

# Lines
lines.linewidth:       2
lines.markersize:      7
lines.markeredgewidth: 2
    
# Legend
legend.frameon:        False
legend.handlelength:   1.0     # adjusts the length of the line segment in the legend


Next, we need to tell matplotlib to use our style sheet by running:

In [ ]:
plt.style.use('default_style.mplstyle')

and that is it. Now you are ready to start creating the figure window.

### Creating a figure window and axes with specific sizes
Matplotlib's built-in functions for creating figure windows and one or more axes, such as plt.subplots(), will do the job whenever you just want to plot something very quickly. However, there are two reason why we would like to improve upon these built-in functions:
1. The built-in functions use inches or relative sizes, while we are more comfortable with using cm.
1. The built-in functions place the axes where they think you want them. This is not good enough when producing panels that are supposed to fit together, as we want to define exactly how large the axes is and where it starts and ends.

Below we will define a function that creates a figure window of a given size and that places a given number of axes inside a plotting region within the figure window.

In [ ]:
%%writefile plotting_functions.py
import matplotlib.pyplot as plt

def get_figure_win(fig_size_cm, plot_rect_cm, grid_size=[1, 1], hor_ver_sep_cm=[0, 0]):
    """
    Create a figure window and axes with a specific size and placement.
    
        Arguments:
            fig_size_cm (list): width and height of the figure window in cm [width, height].
            plot_rect_cm (list): rectange defining the plotting region inside the figure 
                                 window in cm [left, bottom, width, height].
        
        Keyword arguments:
            grid_size (list): the number of rows and columsn for the grid of axes [n_rows, n_cols].
            hor_ver_sep_cm (list): horizontal and vertical separation between axes in cm [hor, ver].
    
    """
    # Convert the fig size to inches
    fig_size = [l/2.54 for l in fig_size_cm]
    # Define the plotting rectangle containing the grid with all axis
    # cm values have to be converted to relative units (relative to the figure window)
    plot_rect = [plot_rect_cm[0]/fig_size_cm[0],
                 plot_rect_cm[1]/fig_size_cm[1],
                 plot_rect_cm[2]/fig_size_cm[0],
                 plot_rect_cm[3]/fig_size_cm[1],]
    # Horizontal and vertical separation between axes (in relative units)
    hor_ver_sep = [hor_ver_sep_cm[i]/fig_size_cm[i] for i in range(2)]
    # The size of a single axes (in relative units)
    axes_size = [ (plot_rect[2]-(grid_size[1]-1)*hor_ver_sep[0]) / grid_size[1],
                  (plot_rect[3]-(grid_size[0]-1)*hor_ver_sep[1]) / grid_size[0]]

    # Create the figure window together with all axis in the grid
    fig = plt.figure(figsize=fig_size)
    axs = []
    for row in range(grid_size[0]-1, -1, -1):
        for col in range(grid_size[1]):
            left_tmp = plot_rect[0] + col*(axes_size[0]+hor_ver_sep[0])
            bottom_tmp = plot_rect[1] + row*(axes_size[1]+hor_ver_sep[1])
            ax_rect_tmp = [left_tmp, bottom_tmp, axes_size[0], axes_size[1]]
            axs.append(plt.axes(ax_rect_tmp, facecolor='none'))

    return fig, axs

Now, let's try out or function by plotting a linear and a cubic function on two separate axes placed next to each other.

In [ ]:
# We use the IPython autoreload extension in the notebook here as we want to add functions to 
# the plotting_function module later on and reload it afterwards.
%load_ext autoreload
%autoreload 1
%aimport plotting_functions

from plotting_functions import get_figure_win

# Define the figure window, the plotting region, 
# and that we want two axes on one row with some horizontal separation
fig_size_cm = [16, 5]                     # [width, height]
plot_rect_cm = [1.75, 1.25, 13.75, 2.75]  # [left, bottom, width, height]
n_subfigs = [1, 2]                        # [n_rows, n_cols]
hor_ver_sep_cm = [2, 0]                   # [hor, ver]

# Extract the colors from the default color cycle
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']

# Get the figure window and the axes
fig, axs = get_figure_win(fig_size_cm, plot_rect_cm, n_subfigs, hor_ver_sep_cm)

# Create x-data for our fucntions
x_lim = np.array([-4, 4])
x = np.linspace(x_lim[0], x_lim[1], 101)

# loop over both axes and plot the functions
titles = ['Linear', 'Cubic']
for i in range(len(axs)):
    y = x**(2*i+1)
    axs[i].plot(x, y, '-', c=colors[i], label='$y=x^{:}$'.format(2*i+1)) # text enclosed by dollar signs are interpreted as LaTeX
    axs[i].set(xlabel='x', ylabel='y', title=titles[i])
    axs[i].legend()

# Use vector graphics (pdf) when possible.
# Otherwise png or tiff are good raster image choices that both support transparency.
# Make sure to use at least dpi=300 with raster images to ensure good quality.
fig.savefig(figure_panel_dir + os.sep + 'LinearAndCubicFunctions.pdf')           # Vector graphics
fig.savefig(figure_panel_dir + os.sep + 'LinearAndCubicFunctions.png', dpi=300)  # Raster image

**NOTE!** the figure shown above is not exactly what was saved. The backend used for the notebook will do it's best to make sure everything is shown in the notebook, whereas the backend used with savefig saves the figure as you defined it. Meaning, if you for example define the plotting region to be larger than the figure window then you will only see the part that fits inside the figure window in the saved file.

**Additional side note.** Vector graphics is generally preferable to ensure good quality, but Microsoft products, for example Word, don't always let you import figures in pdf format, in which case you might have to use raster images if constrained to work with such softwares. Also, vector graphics can always be converted to raster images easily, while the reverse is not true. you should therefore try to keep figures and panels in vector formats for as long as possible.

Before moving on, verify that you can find the saved matplotlib figure in both formats (pdf and png), and also try to verify that they have the expected size. 

Next, lets define a function that will add labels (a, b, c, etc.) to our axes and append it to the 
plotting_functions module.

In [ ]:
%%writefile -a plotting_functions.py

def add_label_to_axes(ax, label):
    """ 
    Add a label to the upper left corner of an axes
    
        Arguments:
            ax (axes): the axes above which the labels should appear.
            label (str): the label to print
    """
    # Specify the label position from the top left corner of the axes
    label_pos_cm = [-1.2, 0.4] 
    
    # Get the figure size in cm and convert the label pos to relative units
    fig_size_cm = ax.figure.get_size_inches()*2.54
    label_pos = [label_pos_cm[i] / fig_size_cm[i] for i in range(2)]
    
    # Define the location for the label and print it
    ax_rect = ax.get_position()
    x_pos = ax_rect.x0 + label_pos[0]
    y_pos = ax_rect.y1 + label_pos[1]
    ax.figure.text(x_pos, y_pos, label, weight='bold', fontsize='large')

Import the new function and save a new version of our figure with labels included.

In [ ]:
# This will not work without the autoreload magic defined earlier
from plotting_functions import add_label_to_axes

labels = ['a', 'b']
for i in range(len(axs)):
    add_label_to_axes(axs[i], labels[i])
        
fig.savefig(figure_panel_dir + os.sep + 'LinearAndCubicFunctionsWithLabels.pdf')

As before, ensure that you can find the new figure and that it looks as expected.

Next, lets make a third panel in a separate figure window, but with the same figure size and the same figure plotting regions. We shall see that panels can be combined easily as long as the plotting regions coincide.

In [ ]:
from scipy.interpolate import interp1d

# Lets use the same figure size and plotting region
# We only redefine the n_subfigs so that we now only get one wide panel
n_subfigs = [1, 1]                        # [n_rows, n_cols]

# Get the figure window and the axes
fig, axs = get_figure_win(fig_size_cm, plot_rect_cm, n_subfigs)

# Let's do linear and cubic interpolation between data points obtained at specific time intervals
# First, we generate observations (randomly)
n_observations = 11
t = np.arange(n_observations)
np.random.seed(5)
observations = np.random.rand(n_observations)
# The, wed define denser time points for our interpolations
t_dense = np.linspace(0, n_observations-1, 101)
# and perform linear and cubic interpolation
linear_interp = interp1d(t, observations, kind='linear')
cubic_interp = interp1d(t, observations, kind='cubic')

axs[0].plot(t, observations, 'ko', mfc='k', alpha=0.3, label='Data')
axs[0].plot(t_dense, linear_interp(t_dense), '-', label='Linear interp.')
axs[0].plot(t_dense, cubic_interp(t_dense), '-', label='Cubic interp.')
axs[0].set(xlabel='Time (s)', ylabel='Observed value', title='Interpolation methods')
axs[0].legend()

add_label_to_axes(axs[0], 'c')
fig.savefig(figure_panel_dir + os.sep + 'InterpolationMethods.pdf')

# Part 2, combining figure panels using TikZ (LaTeX)
Your "Figure panels" directory should now contain two figure panels (or matplotlib figures: LinearAndCubicFunctionsWithLabels.pdf and InterpolationMethods.pdf). Now, we will use TikZ to combine both into one complete figure and add an additional graphical element to illustrate what TikZ can do.

TikZ, lacking a GUI, definitely takes longer to master than many other softwares, and the [manual](https://www.nic.funet.fi/pub/TeX/CTAN/graphics/pgf/base/doc/pgfmanual.pdf) is quite extensive to say the least (there also exist a shorter [minimal introduction](http://cremeronline.com/LaTeX/minimaltikz.pdf)), Luckily though, simple things like combining panels into a complete figure are relatively easy. The most important thing for learning and developing your skills is to have a working template to start from, and that is precisely what we will create here. 

First we will define and save a file with general TikZ settings and styles (similar to the matplotlib style sheet). It is strictly not necessary to make a separate file in this case as we only intend to make one figure, but publications typically have more than one figure and then this is useful to ensure that all definitions are located in one place only. Also, OBSERVE that the files we create here are tex files, meaning that the percent sign '%' is used to denote comments.

In [ ]:
%%writefile Tikz/TikzSettings.tex

% Packages to use (math symbols and helvetica font).
\usepackage{amsmath,amssymb}
\usepackage[scaled]{helvet}
\renewcommand\familydefault{\sfdefault} 
\usepackage{sfmath}

% Define your own colors.
\definecolor{lightGray}{rgb}{0.8, 0.8, 0.8}

% Specify useful parameters.
\newcommand{\unit}{1mm}
\newcommand{\lineWidth}{2pt}

% Specify styles.
% Define a node style for figure panels that are of exactly the same size as the panel.
\tikzstyle{figPanel}=[draw=none, inner sep=0, outer sep=0]
% Define a node style for text boxes, where the box has rounded corners ans is slightly larger that the text.
\tikzstyle{textNode}=[draw=none, fill=lightGray, inner sep=1mm, outer sep=0, rounded corners=1mm]
% Define a line style for drawing things
\tikzstyle{lines}=[line width=\lineWidth, rounded corners=1mm, draw=lightGray]

Next, we define and save the TikZ file that creates the complete figure. This is essentially an empty LaTeX document containing nothing but our figure. The actual figure is defined between begin{tikzpicture} and end{tikzpicture}. The logic for the commands below are as follows:
- We create two nodes (with the figPanel style) and import our saved matplotlib panels into each one.
- The first node is given the name "a_and_b" while the second is named "c".
- The second is placed right below the first one by fixing its anchor at its middle top part (i.e. north) and then placing the anchor at the middle bottom part of the first node (i.e. a_and_b.south). Feel free to play around with east, west, north east, etc. to see how it works.
- We place a text box on top the first figure panel (a_and_b.north) add use the draw command to draw an outline around the whole thing.

In [ ]:
%%writefile Tikz/CompleteFigure.tex
\documentclass[tikz, 12pt]{standalone}
\input{TikzSettings.tex}

\begin{document}
\begin{tikzpicture}[x=\unit, y=\unit]

% Place both figure panels on top of each other
\node[figPanel] (a_and_b) at (0, 0) {\includegraphics[]{"../Figure panels/LinearAndCubicFunctionsWithLabels"}};
\node[figPanel, anchor=north] (c) at (a_and_b.south) {\includegraphics[]{"../Figure panels/InterpolationMethods"}};

% Add a frame with a title to highlight that the panels where combined using Tikz
\node[textNode, anchor=south] (fig_title) at (a_and_b.north) {\large Figure panels combined using Tikz};
\draw[lines] (fig_title.east) -- (a_and_b.north east |- fig_title) -- (c.south east) -- (c.south west) -- (a_and_b.north west |- fig_title) -- (fig_title.west);

\end{tikzpicture}
\end{document}

Finally, we call pdflatex (this is why need a LaTeX installation) to create the figure. The notebook will let you run bash commands using cell magic (%%bash), so the final figure will be created by running the cell below.

In [ ]:
%%bash
# Move to the Tikz directory
cd Tikz
# Compile the Tikz figure using pdflatex
pdflatex CompleteFigure.tex
# Delete auxilliary files created by pdflatex
rm *.log
rm *.aux

Now, you should have a figure named "CompleteFigure.pdf" inside the Tikz folder.

### Some final comments on font sizes in LaTeX
By default, TikZ will use the font size specified by the documentclass command (located in "CompleteFigure.tex"), and similar to matplotlib, you can then use additional commands to temporarily change the relative font size. The final size for each command is given by the table below (check this [StackExchange](https://tex.stackexchange.com/questions/24599/what-point-pt-font-size-are-large-etc) answer for more info).

|Command       | 10 pt    | 11 pt     | 12 pt    |
|--------------|----------|-----------|----------|
|\tiny         | 5 pt     | 6 pt      | 6 pt     |
|\scriptsize   | 7 pt     | 8 pt      | 8 pt     |
|\footnotesize | 8 pt     | 9 pt      | 10 pt    |
|\small        | 9 pt     | 10 pt     | 10.95 pt |
|\normalsize   | 10 pt    | 10.95 pt  | 12 pt    |
|\large        | 12 pt    | 12 pt     | 14.4 pt  |
|\Large        | 14.4 pt  | 14.4 pt   | 17.28 pt |
|\LARGE        | 17.28 pt | 17.28 pt  | 20.74 pt |
|\huge         | 20.74 pt | 20.74 pt  | 24.88 pt |
|\Huge         | 24.88 pt | 24.88 pt  | 24.88 pt |

In our case, the text inside the gray box (with relative size \\large) will thus have a font size of 12 pt.

# Additional suggestions for larger projects
Below follows a few additional suggestions when creating more complex figures:
1. Turn the code that produces each matplotlib panel into functions, and create a script that updates all figures in one go.
1. Move all additional plotting parameters (parameters not included in the style sheet), such as the figure size, the plotting regions, and project specific colors to a separate configuration file (or somewhere else where it can be accessed by all panel plotting functions).

The underlying idea being that you should be able to update for example the size of all panels belonging to one particular figure, whenever you decide to change the size of the whole figure. Ideally, you should accomplish this by only changing the figure size in a configuration file, and then running a script that 1) calls all functions responsible for plotting the panels, and 2) runs pdflatex to assemble the updated panels using TikZ.

At first sight, it might feel like an overkill to put this much emphasis on being able to easily update all the figures in your publication. However, during the final stages of any project, there always tends to be numerous fine-tunings of data preprocessing, data analysis, and also the figures themselves. Thus, making sure that all your figures are easily updated will save a lot of time in the end.